<a href="https://colab.research.google.com/github/AmoolyaReddyy/lane-detection/blob/main/U_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import random
import cv2
import math
import matplotlib.pyplot as plt
import os

print(tf.__version__)

2.12.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Input, BatchNormalization, Activation, Add
from tensorflow.keras.layers import AveragePooling2D, Flatten, Conv2DTranspose, concatenate, MaxPool2D, SeparableConv2D,Cropping2D,UpSampling2D

In [ ]:
def _parse_image_function(example_proto):
    """Mapping function for parsing images and annotations from the tfrecord files. Transforms and augments images and annotations before training.
        Args:
            example_proto: a single element from the dataset
        Returns:
            image: transformed image extracted from the datatset
            annotation: transformed annotation extracted from the datatset
    """

    #This descibes the structure of each element in the dataset
    image_feature_description={
        'height':tf.io.FixedLenFeature([], tf.int64),
        'width': tf.io.FixedLenFeature([], tf.int64),
        'image_raw': tf.io.FixedLenFeature([], tf.string),
        'mask_raw': tf.io.FixedLenFeature([], tf.string)
        }

    #The features are extracted to a dictionary
    feature=tf.io.parse_single_example(example_proto, image_feature_description)
    #Images and Annotations are resized to (192,192)
    image = tf.image.decode_jpeg(feature['image_raw'])
    image = tf.cast(image, tf.float32) / 255.0
    annotation = tf.image.decode_png(feature['mask_raw'], channels=1)
    annotation = tf.cast(annotation, tf.float32) / 1.0

    image = tf.reshape(image, (360,640,3))
    annotation = tf.reshape(annotation, (360,640,1))

    image = tf.image.pad_to_bounding_box(image, 140, 0, 640, 640)
    annotation = tf.image.pad_to_bounding_box(annotation, 140, 0, 640, 640)

    image = tf.image.resize(image, size=(192,192))
    annotation = tf.image.resize(annotation, size=(192,192))

    #Randomly flips images and annotations
    if(random.random() > 0.5):
        image = tf.image.flip_left_right(image)
        annotation = tf.image.flip_left_right(annotation)


    return image, annotation

In [ ]:
BATCH_SIZE = 8

train_dataset = tf.data.TFRecordDataset(['/content/drive/My Drive/RUGVED/Lane-Extraction-from-Grass-Surfaces-master/Dataset/tfrecords/train.tfrecords'])
train_dataset = train_dataset.map(_parse_image_function)
ds_train = train_dataset.shuffle(buffer_size=584)
ds_train = ds_train.repeat()
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

val_dataset = tf.data.TFRecordDataset('/content/drive/My Drive/RUGVED/Lane-Extraction-from-Grass-Surfaces-master/Dataset/tfrecords/val.tfrecords')
val_dataset = val_dataset.map(_parse_image_function)
ds_val = val_dataset.shuffle(buffer_size=32)
ds_val = ds_val.repeat()
ds_val = ds_val.batch(BATCH_SIZE)
ds_val = ds_val.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

test_dataset = tf.data.TFRecordDataset('/content/drive/My Drive/RUGVED/Lane-Extraction-from-Grass-Surfaces-master/Dataset/tfrecords/test.tfrecords')
test_dataset = test_dataset.map(_parse_image_function)
ds_test = test_dataset.shuffle(buffer_size=34)
ds_test = ds_test.repeat()
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


In [ ]:
def u_net():
  #Encoder
  input_image = Input(shape=(192,192,3), name='input')
  c1=Conv2D(64,3,activation='relu',padding='same',kernel_initializer='he_normal')(input_image)
  c1=Conv2D(64,3,activation='relu',padding='same',kernel_initializer='he_normal')(c1)
  p1=MaxPool2D(pool_size=(2,2),strides=2)(c1)

  c2=Conv2D(128,3,activation='relu',padding='same',kernel_initializer='he_normal')(p1)
  c2=Conv2D(128,3,activation='relu',padding='same',kernel_initializer='he_normal')(c2)
  p2=MaxPool2D(pool_size=(2,2),strides=2)(c2)

  c3=Conv2D(256,3,activation='relu',padding='same',kernel_initializer='he_normal')(p2)
  c3=Conv2D(256,3,activation='relu',padding='same',kernel_initializer='he_normal')(c3)
  p3=MaxPool2D(pool_size=(2,2),strides=2)(c3)


  c4=Conv2D(512,3,activation='relu',padding='same',kernel_initializer='he_normal')(p3)
  c4=Conv2D(512,3,activation='relu',padding='same',kernel_initializer='he_normal')(c4)
  p4=MaxPool2D(pool_size=(2,2),strides=2)(c4)

  c5=Conv2D(1024,3,activation='relu',padding='same',kernel_initializer='he_normal')(p4)
  c5=Conv2D(1024,3,activation='relu',padding='same',kernel_initializer='he_normal')(c5)
  #Decoder
  up5 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(c5))

  s=int((c4.shape[1]-up5.shape[1])/2);
  cc4=Cropping2D(cropping=((s, s), (s, s)), data_format=None)(c4)
  m1 = concatenate([cc4,up5], axis = 3)


  c6=Conv2D(512,3,activation='relu',padding='same',kernel_initializer='he_normal')(m1)
  c6=Conv2D(512,3,activation='relu',padding='same',kernel_initializer='he_normal')(c6)
  up6 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(c6))

  s=int((c3.shape[1]-up6.shape[1])/2)
  cc3=Cropping2D(cropping=((s, s), (s, s)), data_format=None)(c3)
  m2 = concatenate([cc3,up6], axis = 3)


  c7=Conv2D(256,3,activation='relu',padding='same',kernel_initializer='he_normal')(m2)
  c7=Conv2D(256,3,activation='relu',padding='same',kernel_initializer='he_normal')(c7)
  up7 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(c7))

  s=int((c2.shape[1]-up7.shape[1])/2);
  cc2=Cropping2D(cropping=((s, s), (s, s)), data_format=None)(c2)
  m3 = concatenate([cc2,up7], axis = 3)

  c8=Conv2D(128,3,activation='relu',padding='same',kernel_initializer='he_normal')(m3)
  c8=Conv2D(128,3,activation='relu',padding='same',kernel_initializer='he_normal')(c8)
  up8 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(c8))

  s=int((c1.shape[1]-up8.shape[1])/2);
  cc1=Cropping2D(cropping=((s, s), (s, s)), data_format=None)(c1)
  m4 = concatenate([cc1,up8], axis = 3)

  c9=Conv2D(64,3,activation='relu',padding='same',kernel_initializer='he_normal')(m4)
  c9=Conv2D(64,3,activation='relu',padding='same',kernel_initializer='he_normal')(c9)
  op = Conv2D(1, 1, activation = 'sigmoid', padding = 'same', kernel_initializer = 'he_normal')(c9)
  model = Model(inputs=[input_image], outputs=[op])
  return model






In [ ]:
model=u_net()
model.summary()

In [ ]:
def step_decay(epoch):

    initial_lrate = 0.001
    drop = 0.9
    epochs_drop = 2
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

In [ ]:
lrate = LearningRateScheduler(step_decay)
class myCallBack(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') >= 0.995):
            self.model.stop_training = True

callback = myCallBack()

checkpoint_path = "/content/drive/My Drive/RUGVED/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
callbacks_list = [lrate, callback,cp_callback]

In [ ]:
iterator = iter(ds_train)
iterator1 = iter(ds_val)
iterator2=iter(ds_test)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.0), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(iterator,steps_per_epoch=600//BATCH_SIZE,epochs=20, validation_data=iterator1, validation_steps=1, callbacks=callbacks_list, verbose = 1)

NameError: ignored

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()


NameError: ignored

In [ ]:
model.evaluate(iterator2,batch_size=16, steps=20,verbose = 1)

In [ ]:
model.save('funet.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image as im
from tensorflow.keras.models import Model,load_model
model = load_model('/content/drive/My Drive/RUGVED/funet.h5')

In [ ]:
img_path='/content/drive/My Drive/RUGVED/testing_images/image_5.webp'

In [ ]:
!ls '/content/drive/My Drive/RUGVED'

In [ ]:


def predict(img_path):
    img = image.load_img(img_path, target_size=(192,192))
    img_array = image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    img_preprocessed = img_batch
    mask = model.predict([[img_preprocessed]])
    mask = np.resize(mask, (192,192))
    #mask = np.round(mask)
    mask = mask * 255

    '''print(mask[(np.where(mask >= 0.5)) and (np.where(mask <1))].size)'''
    fig = plt.figure(figsize=(10, 7))
    rows = 1
    columns = 2
    fig.add_subplot(rows, columns, 1)
    plt.imshow(img )
    plt.title("Input_image")
    fig.add_subplot(rows, columns, 2)
    plt.imshow(mask,cmap='Greys_r')
    plt.title("Mask")

predict(img_path)


NameError: ignored